# Meta-RL Predictability Study: Stock-Month Advantage Classifier

## Objective

Identify and classify stock-month environments where a reinforcement learning (RL) agent (PPO) outperforms a random strategy. These environments are considered "predictable" or advantageous for training trading agents.

---

## Dataset

* **Source:** Cleaned OHLCV data (daily resolution)
* **Date range:** 2022-01-01 to 2023-01-01
* **Universe:** Top US equities (e.g., AAPL, MSFT, JPM, AMZN, TSLA, etc.)
* **Exclusions:** Tickers with missing or corrupted data

---

##  Meta-Feature Engineering

For each stock-month pair (symbol, month):

1. **Return Distribution** from month *t*:

   * Mean Return
   * Std Return
   * Skewness
   * Kurtosis
   * Shannon Entropy (10-bin histogram)

2. **Volume Features**:

   * Mean Volume
   * Std Volume

3. **Residual Diagnostics (month t+1)**:

   * Fit `RandomForestRegressor` on lagged returns (lags=5)
   * Compute residuals: `actual - predicted`
   * Ljung-Box p-value (autocorrelation)
   * Residual ACF(1)
   * Residual Std, Skew, Kurtosis

4. **Rolling R² (CV)**

   * Use 3-fold cross-validation on the same model
   * Target: `cv_r2` as a proxy for local predictability

---

## Meta-RL Labeling Pipeline

1. **Custom Gym Environment**: `CumulativeTradingEnv`
2. **Per stock-month**:

   * Train PPO agent (stable-baselines3)
   * Evaluate cumulative reward
   * Compare against baseline (random policy)
   * Compute:

     * `agent_reward`, `random_reward`
     * `advantage = agent_reward - random_reward`
     * Other metrics: `sharpe`, `alpha`, `cumulative_return`
3. **Label Definition**:

   * Binary target: `1` if advantage > 0 else `0`

---

## Final Feature Set

```
['mean_return', 'std_return', 'skew', 'kurtosis', 'entropy',
 'vol_mean', 'vol_std',
 'ljung_pval', 'resid_acf1', 'resid_std', 'resid_skew', 'resid_kurtosis',
 'sharpe', 'cum_return', 'alpha']
```

---

## Classifier Results

**Model**: `RandomForestClassifier(n_estimators=200, class_weight="balanced")`

**Train/Test Split**: 80/20 (Stratified)

### Classification Report:

```
              precision    recall  f1-score   support

           0       0.84      0.95      0.89       420
           1       0.97      0.89      0.93       675

    accuracy                           0.91      1095
   macro avg       0.91      0.92      0.91      1095
weighted avg       0.92      0.91      0.91      1095
```

### Confusion Matrix:

```
[[400  20]
 [ 74 601]]
```

---

## Interpretation

* High overall accuracy (91%) and balanced class performance
* The model is especially good at predicting when RL **will** work (class 1)
* A powerful tool for **pre-filtering environments** before launching full RL training

---

## Next Steps

* ✅ Feature importance (SHAP, permutation)
* ✅ Add volatility-based meta-features (GARCH, Hurst exponent)
* ✅ Try ranking models: `A > B` if `adv_A > adv_B`
* ✅ Out-of-time validation on 2023+ data
* ✅ Use this model to control which environments enter RL portfolios

---

## Files

* `features_<EXPERIENCE_NAME>.pkl`: Engineered features
* `targets_<EXPERIENCE_NAME>.pkl`: CV R² scores
* `meta_rl_labels_<EXPERIENCE_NAME>.pkl`: Advantage labeling

---

## Requirements

* `scikit-learn`
* `statsmodels`
* `stable-baselines3`
* `gymnasium`
* `pandas`, `numpy`, `joblib`

---

## Authors

* Filipe Mota de Sá @ github.io/pihh
* ChatGPT (OpenAI)

---

*"Not all environments are created equal. Filter wisely."*


In [1]:
# SETUP: Imports & Paths ===========================
import jupyter
from src.utils.system import boot, Notify

boot()
import os
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from tqdm import tqdm

from src.data.feature_pipeline import basic_chart_features,load_base_dataframe
from src.predictability.easiness import rolling_sharpe, rolling_r2, rolling_info_ratio, rolling_autocorr
from src.predictability.pipeline import generate_universe_easiness_report
from IPython import display

from src.experiments.experiment_tracker import ExperimentTracker
from src.config import TOP2_STOCK_BY_SECTOR


from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from scipy.stats import skew, kurtosis, entropy
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf, acovf


import warnings
warnings.filterwarnings("ignore")


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [42]:

EXPERIENCE_NAME = "stock_universe_predictability_selection__MetaFeatures__MetaRlLabeling"
FEATURES_PATH = f"../data/cache/features_{EXPERIENCE_NAME}.pkl"
TARGETS_PATH = f"../data/cache/targets_{EXPERIENCE_NAME}.pkl"
META_PATH = f"../data/cache/meta_{EXPERIENCE_NAME}.pkl"

excluded_tickers=['CEG', 'GEHC', 'GEV', 'KVUE', 'SOLV']
excluded_tickers.sort()
#tickers = TOP2_STOCK_BY_SECTOR

config={
    "regressor":"RandomForestRegressor",
    "n_estimators": 200,
    "random_state":314,
    "transaction_cost":0
}
run_settings={
    "excluded_tickers": excluded_tickers,
    "min_samples": 10,
    "cv_folds": 3,
    "lags": 5,
    "start_date":"2022-01-01",
    "end_date":"2025-01-01",
    "seed":314,
    "episode_length":18,
    "noise_feature_cols": ["return_1d", "volume"]  ,

    "train_steps": 300,
    "min_ep_len" : 18
}

# Config section


In [3]:
# LOAD OHLCV ==========================================


ohlcv_df = load_base_dataframe()
ohlcv_df['date'] = pd.to_datetime(ohlcv_df['date'])
ohlcv_df = ohlcv_df[(ohlcv_df['date'] >= run_settings["start_date"]) & (ohlcv_df['date'] < run_settings["end_date"])]
ohlcv_df['month'] = ohlcv_df['date'].dt.to_period('M')
ohlcv_df['return_1d'] = ohlcv_df['return_1d'].fillna(0)
ohlcv_df['sector_id'] = ohlcv_df['sector_id'].fillna('unknown')
ohlcv_df['industry_id'] = ohlcv_df['industry_id'].fillna('unknown')

In [32]:
# Meta-feature & Label Extraction =======================

tickers = ohlcv_df['symbol'].unique()
tickers = tickers[~np.isin(tickers, excluded_tickers)]
def mean_policy(arr):
    # return np.median(arr)
    return pd.Series(arr).ewm(span=5).mean().iloc[-1]

# Attempt to load if already exists (resumability)
if all([os.path.exists(path) for path in [FEATURES_PATH, TARGETS_PATH, META_PATH]]):
    features = joblib.load(FEATURES_PATH)
    targets = joblib.load(TARGETS_PATH)
    metadata = joblib.load(META_PATH)
    print("Loaded cached feature/target/meta lists.")
    
else:
    features, targets, metadata = [], [], []
    #tickers = ohlcv_df['symbol'].unique()
    #tickers = [t for t in tickers if t not in run_settings["excluded_tickers"]]
    for symbol in tqdm(tickers):
        df = ohlcv_df[ohlcv_df['symbol'] == symbol].sort_values('date').copy()
        months = df['month'].unique()
        for i in range(1, len(months)):
            m_t = months[i-1]
            m_t1 = months[i]
            df_t = df[df['month'] == m_t]
            df_t1 = df[df['month'] == m_t1]
            if len(df_t1) < run_settings["min_samples"]:
                continue
            r1d = df_t['return_1d'].astype(float).values
            v = df_t['volume'].astype(float).values
            feat = {
                'symbol': symbol,
                'month_str': str(m_t),
                'mean_return': mean_policy(r1d),
                'std_return': r1d.std(),
                'skew': skew(r1d),
                'kurtosis': kurtosis(r1d),
                'entropy': entropy(np.histogram(r1d, bins=10, density=True)[0] + 1e-8),
                'vol_mean': mean_policy(v),
                'vol_std': v.std()
            }
            # Residual diagnostics from simple RF on t+1
            df_lag = df_t1.copy()
            for lag in range(1, run_settings['lags'] + 1):
                df_lag[f'return_lag_{lag}'] = df_lag['return_1d'].shift(lag)
            df_lag = df_lag.dropna()
            if len(df_lag) < run_settings["min_samples"]:
                continue
            X = df_lag[[f'return_lag_{i}' for i in range(1, run_settings['lags'] + 1)]].values
            y = df_lag['return_1d'].values
            model = RandomForestRegressor(n_estimators=config['n_estimators'], random_state=config['random_state'])
            model.fit(X, y)
            residuals = y - model.predict(X)
            # Meta-diagnostics
            ljung_pval = acorr_ljungbox(residuals, lags=[run_settings['lags']], return_df=True).iloc[0]['lb_pvalue']
            feat['ljung_pval'] = ljung_pval
            feat['resid_acf1'] = pd.Series(residuals).autocorr(lag=1)
            feat['resid_std'] = residuals.std()
            feat['resid_skew'] = skew(residuals)
            feat['resid_kurtosis'] = kurtosis(residuals)
            # Predictability label (cross-val R²)
            cv_r2 = mean_policy(cross_val_score(model, X, y, cv=run_settings["cv_folds"], scoring='r2'))
            features.append(feat)
            targets.append(cv_r2)
            metadata.append((symbol, str(m_t)))
    # Save for future resumes
    joblib.dump(features, FEATURES_PATH)
    joblib.dump(targets, TARGETS_PATH)
    joblib.dump(metadata, META_PATH)
    print("Feature/target/meta lists saved.")


100%|██████████| 499/499 [2:58:40<00:00, 21.48s/it]  


Feature/target/meta lists saved.


In [33]:
# DataFrame Construction  ============================
X_df = pd.DataFrame(features)
y_df = pd.Series(targets, name='cv_r2')
meta_df = pd.DataFrame(metadata, columns=['symbol', 'month'])

In [34]:
# Scaling & Preparation ==============================

X = X_df.drop(columns=['symbol', 'month_str'])
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
X_df.sort_values(by="month_str")

,symbol,month_str,mean_return,std_return,skew,kurtosis,entropy,vol_mean,vol_std,ljung_pval,resid_acf1,resid_std,resid_skew,resid_kurtosis
0,MMM,2022-01,-0.005358,0.014943,-0.917692,0.261870,1.955084,5.306873e+06,1.376213e+06,0.886434,-0.210568,0.011435,1.832094,3.398314
11305,MS,2022-01,0.006825,0.021745,-0.333840,-0.221729,1.858225,1.422368e+07,5.778826e+06,0.715527,-0.154054,0.010775,0.159432,-0.630037
1225,ANSS,2022-01,0.019428,0.027280,0.757380,-0.219618,2.159732,7.332955e+05,1.487807e+05,0.845148,0.233395,0.013110,0.247074,-0.517630
11270,MCO,2022-01,0.010028,0.020841,-0.223985,-0.291958,1.767378,1.036493e+06,3.342355e+05,0.309837,0.112161,0.006384,-0.811815,0.007825
11235,MNST,2022-01,0.005772,0.019329,0.146634,-0.296322,1.937200,2.933086e+06,6.461404e+05,0.995052,0.116723,0.010972,1.322396,2.791476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9589,KLAC,2024-11,0.007148,0.018601,-0.755084,0.190229,1.752424,1.084534e+06,3.136550e+05,0.642694,-0.093129,0.006706,-0.542184,-0.551728
12774,PEP,2024-11,0.003046,0.011626,-1.847074,3.740503,1.690831,4.487059e+06,1.727644e+06,0.041658,-0.311018,0.004098,0.282088,0.332343
6264,EVRG,2024-11,-0.000004,0.008593,0.149888,-0.165370,1.946839,1.394257e+06,5.842667e+05,0.214443,0.053386,0.003162,1.147279,2.221280
12914,PSX,2024-11,0.002358,0.012867,1.876282,4.872534,1.608883,2.259777e+06,6.141416e+05,0.344137,0.073886,0.008953,-0.500015,-0.242101


In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import joblib
from src.env.base_trading_env import CumulativeTradingEnv
RL_LABELS_PATH = f"../../data/cache/meta_rl_labels_{EXPERIENCE_NAME}.pkl"

feature_cols = run_settings["noise_feature_cols"] # ["return_1d", "volume"]  # Or your preferred features
episode_length = run_settings["episode_length"] # 18  # Or whatever fits your month
train_steps = run_settings["train_steps"] # 300    # Fast!
min_ep_len = run_settings["min_ep_len"] # 18
# Resume logic: Load meta_df with RL columns if available
if os.path.exists(RL_LABELS_PATH):
    meta_df_rl = pd.read_pickle(RL_LABELS_PATH)
    print("Loaded meta_df with RL columns.")
else:
    # Copy original meta_df and initialize RL columns
    meta_df_rl = meta_df.copy()
    meta_df_rl['agent_reward'] = np.nan
    meta_df_rl['random_reward'] = np.nan
    meta_df_rl['advantage'] = np.nan
    meta_df_rl['sharpe'] = np.nan
    meta_df_rl['cum_return'] = np.nan
    meta_df_rl['alpha'] = np.nan


for i, row in tqdm(meta_df_rl.iterrows(), total=len(meta_df_rl), desc="Meta-RL Agent Loop"):
    # Skip if already computed
    if not np.isnan(meta_df_rl.loc[i, 'agent_reward']):
        continue

    symbol, month = row['symbol'], row['month']
    df_env = ohlcv_df[(ohlcv_df['symbol'] == symbol) & (ohlcv_df['month'] == month)].sort_values("date")
    if len(df_env) < min_ep_len:
        min_ep_len = len(df_env)
        print('new min',min_ep_len)
    if len(df_env) < episode_length:
        print('x',len(df_env) ,episode_length)
        continue  # Not enough data, skip

    try:
        env = CumulativeTradingEnv(
            df=df_env,
            feature_cols=feature_cols,
            episode_length=episode_length,
            transaction_cost=config['transaction_cost'],#0.0001,
            seed=run_settings['seed']
        )
        env = gym.wrappers.FlattenObservation(env)
        check_env(env, warn=True)

        model = PPO("MlpPolicy", env, verbose=0, n_steps=64, batch_size=16, learning_rate=0.001, seed=run_settings['seed'])
        model.learn(total_timesteps=train_steps)

        # Evaluate PPO
        obs, _ = env.reset()
        agent_rewards, done = [], False
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(action)
            agent_rewards.append(reward)
        agent_reward = np.sum(agent_rewards)

        # Evaluate Random
        obs, _ = env.reset()
        random_rewards, done = [], False
        while not done:
            action = env.action_space.sample()
            obs, reward, done, truncated, info = env.step(action)
            random_rewards.append(reward)
        random_reward = np.sum(random_rewards)

        advantage = agent_reward - random_reward

        meta_df_rl.loc[i, 'agent_reward'] = agent_reward
        meta_df_rl.loc[i, 'random_reward'] = random_reward
        meta_df_rl.loc[i, 'advantage'] = advantage
        meta_df_rl.loc[i, 'sharpe'] = info.get("episode_sharpe", np.nan)
        meta_df_rl.loc[i, 'cum_return'] = info.get("cumulative_return", np.nan)
        meta_df_rl.loc[i, 'alpha'] = info.get("alpha", np.nan)
        #print(info)
        
    except Exception as e:
        print(f"Skipped ({symbol})",e)


Meta-RL Agent Loop:   0%|          | 0/17441 [00:00<?, ?it/s]

['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']


Meta-RL Agent Loop:   0%|          | 1/17441 [00:02<13:01:30,  2.69s/it]

['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']
['MMM']


Meta-RL Agent Loop:   0%|          | 2/17441 [00:05<12:31:04,  2.58s/it]

...

['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']


Meta-RL Agent Loop:   9%|▉         | 1622/17441 [56:47<8:58:46,  2.04s/it]

['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']


Meta-RL Agent Loop:   9%|▉         | 1623/17441 [56:48<8:41:20,  1.98s/it]

['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']
['AIZ']


In [ ]:
meta_df_rl.to_csv('mrl.csv')
meta_df_rl['target'] = (meta_df_rl['advantage'] > 0).astype(int)
meta_df_rl

In [ ]:
feature_cols = [col for col in meta_df_rl.columns if col not in ['symbol', 'month', 'agent_reward', 'random_reward', 'advantage', 'target']]
feature_cols

In [ ]:
meta_df_rl['target'].value_counts()

In [ ]:
# Make sure columns are compatible for merge
X_df['month'] = X_df['month_str']
merged = pd.merge(X_df, meta_df_rl, on=['symbol', 'month'], how='inner')
merged

In [ ]:
feature_cols = [
    col for col in merged.columns
    if col not in ['symbol', 'month', 'month_str', 'agent_reward', 'random_reward', 'advantage', 'target']
]

X = merged[feature_cols]
y = merged['target']

# Scale features
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
feature_cols

In [ ]:
#ohlcv_df.sort_values(by="date").head().to_csv('ohlcv_to_upload.csv')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=run_settings['seed']
)

clf = RandomForestClassifier(n_estimators=config['n_estimators'], random_state=run_settings['seed'], class_weight='balanced')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

importances = clf.feature_importances_
sorted_idx = np.argsort(importances)[::-1]
plt.figure(figsize=(12, 6))
plt.bar(range(len(importances)), importances[sorted_idx])
plt.xticks(range(len(importances)), [feature_cols[i] for i in sorted_idx], rotation=90)
plt.title("Meta-Feature Importances for Predicting RL Agent Advantage")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# y_true: true labels, y_pred: predicted labels
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)  # Optional: color map
plt.title("Confusion Matrix")
plt.show()


class_names = ['Will Learn', 'Wont learn']  # Adjust to your problem

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Oranges)
plt.title("Confusion Matrix")
plt.show()

## Quick Recap
We want to estimate how "predictable" each stock is in a given month, using meta-features of its behavior.

#### Pipeline: 
**Loop: For each (stock, month)**
From Previous Month (t) we will extract features. From the returns in month t, we compute:
* Mean
* Std 
* Skew 
* Kurtosis
* Entropy of returns
* Mean of volumne
* Std of volume

These become the meta-features for that stock-month.

**From Following Month (t+1) we will compute "predictability"**

* With 5 lags of daily returns from month t+1 will try to predict daily returns using a RandomForestRegressor
* Evaluate performance with cross-validated R² (cv_r2)
* Analyze residuals from this model with the Ljung–Box test for autocorrelation ⇒ gives ljung_pval

These become the target labels or diagnostic scores

| Feature                 | Description                                        |
| ----------------------- | -------------------------------------------------- |
| `resid_acf1`            | Autocorrelation of residuals (lag 1)               |
| `resid_std`             | Std of residuals                                   |
| `resid_skew`            | Skewness of residuals                              |
| `resid_kurtosis`        | Kurtosis of residuals                              |
| `resid_ljung_pval`      | p-value of Ljung-Box test for residual autocorr    |
| `return_autocorr_1d`    | Lag-1 autocorrelation of raw 1D returns            |
| `volatility_clustering` | Rolling std autocorrelation (vol clustering proxy) |


In [ ]:
2. Feature engineering:
Add features that measure uncertainty, volatility clustering, or market randomness like:

Hurst exponent

GARCH volatility

Rolling ADF p-values

Change-point detection count

| Stage                 | Description                                                                                             |
| --------------------- | ------------------------------------------------------------------------------------------------------- |
| 🧹 Preprocessing      | Clean stock OHLCV data, compute lagged returns                                                          |
| 📈 Forecast Models    | Run simple regressors on next-month returns                                                             |
| 🔍 Diagnostics        | Extract residual meta-features, R², forecast entropy, etc.                                              |
| 🧠 Labeling           | - Regression: R² as target<br>- Ranking: (A > B)<br>- RL Reward: agent learnability                     |
| 📊 Feature Extraction | Use summary stats + diagnostic/meta features                                                            |
| 🧬 Modeling           | - Regression: Predict R²<br>- Classification: Predict "learnable"<br>- Contrastive: Rank predictability |
| 🏆 Output             | Sorted top-k stock-months or environments where RL thrives                                              |


| Section                             | Purpose                                                                                                                                                                                                          |
| ----------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 🔧 **Setup & Pipeline Description** | High-level explanation of your RL pipeline, feature engineering, and data sources                                                                                                                                |
| 🧪 **Completed Studies**            | Summary table or list of ablation studies, e.g.:<br>`01 - Reward Function Impact`<br>`02 - Predictability Filters via R²`<br>`03 - Meta-Learnability Scores`                                                     |
| ✅ **Conclusions So Far**            | Bullet points of key findings from each experiment, e.g.:<br>– Simple R² doesn't generalize across time<br>– Residual-based features offer better stability<br>– Meta-RL proxy labels correlate with test Sharpe |
| 🔬 **Ongoing Work**                 | One-liner of what’s running or planned, so future you remembers                                                                                                                                                  |
| 📎 **Notebook Index**               | List of notebooks and what each one covers                                                                                                                                                                       |
